In [ ]:
# default_exp getCikFilings

# getCikFilings

> Get filings list for a CIK using the SEC's RESTful API.

In [ ]:
#hide
%load_ext autoreload
%autoreload 2
from nbdev import show_doc

In [ ]:
#export

from secscan import utils

Download and parse a list of filings for a CIK:

In [ ]:
#export

def getCikFilings(cik) :
    restFilingsUrl = f'/submissions/CIK{str(cik).zfill(10)}.json'
    filingsJson = utils.downloadSecUrl(restFilingsUrl, restData=True, toFormat='json')
    return filingsJson

In [ ]:
testJ = getCikFilings(83350)
print(testJ['name'])
testJF = testJ['filings']['recent']
assert {'0001437749-21-013386','0001437749-21-008960'} < set(testJF['accessionNumber'])

RESERVE PETROLEUM CO


In [ ]:
# list(zip(jf['accessionNumber'],jf['filingDate'],jf['form']))

In [ ]:
#hide
# uncomment and run to regenerate all library Python files
# from nbdev.export import notebook2script; notebook2script()